In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

def assign_gen(well): # function to assign genotype to well
    gen=wells_gen_dict.get(well) # get genotype from dictionary
    return gen


In [ ]:
# function to calculate volume of ventricle and atrium
def calculate_vol(file_path): 
    df=pd.read_csv(file_path).set_index("Measurement_Name") # read csv file
    well=df.Label.str.split(":-",expand=True)[1].str.split("--PO",expand=True)[0][0] # get well from file name
    rep=file_path.split("No-")[1].split(".csv")[0] # get replicate from file name
    if "Diastole" in file_path: # if diastole, calculate volume of ventricle and atrium
        vol_ventricle = (((1/6)*math.pi)*(df.loc["ventricle_short"]["Length"])*(df.loc["ventricle_long"]["Length"])**2) # calculate volume of ventricle
        vol_atrium  = (((1/6)*math.pi)*(df.loc["atrium_short"]["Length"])*(df.loc["atrium_long"]["Length"])**2) # calculate volume of atrium
        phase="Diastole" # assign phase
    elif "Systole" in file_path: # if systole, calculate volume of ventricle and atrium
        vol_ventricle = (((1/6)*math.pi)*(df.loc["ventricle_short"]["Length"])*(df.loc["ventricle_long"]["Length"])**2) # calculate volume of ventricle
        vol_atrium  = (((1/6)*math.pi)*(df.loc["atrium_short"]["Length"])*(df.loc["atrium_long"]["Length"])**2) # calculate volume of atrium
        phase="Systole" # assign phase
    volumes=(well,rep,phase,vol_ventricle,vol_atrium) # create tuple with well, replicate, phase, volume of ventricle and volume of atrium
    return volumes # return tuple


In [ ]:
# Define input

folder_path="/AcquiferResults/Ejection_fraction/Measured_diameters/"
df_hb = pd.read_csv("C:/AcquiferResults/Heartbeat/result_heartbeat_B.csv", sep=";").rename(columns={"well":"Well"}) # read csv file with heartbeat data


In [ ]:
# Load ejection fraction tables, make calculation

list_directories=os.listdir(folder_path) # get list of directories
df_volume_phase=pd.DataFrame([calculate_vol(folder_path+i) for i in list_directories],columns=["Well","Repeat","Phase","Volume_Ventricle","Volume_Atrium"]) # calculate volume of ventricle and atrium
df_volume_phase_grouped=df_volume_phase.groupby(by=["Well","Phase"]).mean().reset_index() # group by well and phase
df_diastole=df_volume_phase[df_volume_phase["Phase"]=="Diastole"] # get diastole
df_systole=df_volume_phase[df_volume_phase["Phase"]=="Systole"] # get systole
df_dis_sys=pd.merge(df_diastole,df_systole, on=["Well","Repeat"]).drop(["Phase_x","Phase_y"],axis=1).rename(columns={"Volume_Ventricle_x":"Volume_Ventricle_Diastole","Volume_Atrium_x":"Volume_Atrium_Diastole","Volume_Ventricle_y":"Volume_Ventricle_Systole","Volume_Atrium_y":"Volume_Atrium_Systole"}) # merge diastole and systole

df_dis_sys["Ejection_fraction_Ventricle[%]"]=100*(df_dis_sys["Volume_Ventricle_Diastole"]-df_dis_sys["Volume_Ventricle_Systole"])/df_dis_sys["Volume_Ventricle_Diastole"] # calculate ejection fraction of ventricle


In [ ]:
# Calculate meam for Ejection_fraction_Ventricle[%] > 0 

df_dis_sys_mean=df_dis_sys[df_dis_sys["Ejection_fraction_Ventricle[%]"]>0].groupby("Well").mean().reset_index() # calculate mean for ejection fraction of ventricle > 0
df_dis_sys_mean


In [ ]:
# Assign genotype 

wells = pd.read_csv("C:/AcquiferResults/Heartbeat/result_heartbeat_B.csv", sep=";") # read csv file with heartbeat data
wells_dropna = wells.dropna()
wells_gen = wells_dropna.drop(columns=wells_dropna.columns[0:3]) # drop columns
wells_gen_dict=wells_gen.set_index("well").to_dict()["gen."] # create dictionary with well and genotype
df_dis_sys_mean["gen."]=df_dis_sys_mean["Well"].apply(assign_gen) # assign genotype to well

In [ ]:
df_merged=df_dis_sys_mean.merge(df_hb,on="Well").drop(["gen._x","Image_title","Rating"],axis=1) # merge dataframes
df_merged["Beats_per_min"]=(df_merged["Heartbeat"]/(300*30))*1000*60 # calculate beats per minute
df_merged['Stroke_Volume_Ventricle_nl']=((df_merged.Volume_Ventricle_Diastole)-(df_merged.Volume_Ventricle_Systole))*(10**-9)*1000 # calculate stroke volume of ventricle in nl
df_merged['Cardiac_Output_nl_per_min'] = ((df_merged.Beats_per_min)*(df_merged.Stroke_Volume_Ventricle_nl)) # calculate cardiac output in nl/min
df_merged # show dataframe
df_merged.to_csv("nrs_cardiac function.csv") #  save dataframe as csv file

In [ ]:
# Merge more results

size = pd.read_csv("C:/AcquiferResults/Size/20210519_nrs-3dpf_length-genotyping.csv", delimiter=";") # read csv file with size data
size_dropna = size.dropna() # drop na
size_gen = size_dropna.drop(columns=wells_dropna.columns[0:2]) # drop columns
df_full=df_merged.merge(size_gen,on="Well") # merge dataframes
df_full # show dataframe



In [ ]:
print("Length vs Ventricle volume:", df_full['Length'].corr(df_full['Volume_Ventricle_Diastole'])) # calculate correlation
print("Lenghh vs Atrium volume:", df_full['Length'].corr(df_full['Volume_Atrium_Systole'])) # calculate correlation
print("Length vs Heartbeat:", df_full['Length'].corr(df_full['Heartbeat'])) # calculate correlation
print("Length vs Beats_per_min:", df_full['Length'].corr(df_full['Beats_per_min'])) # calculate correlation
print("Length vs Stroke_Volume_Ventricle_nl:", df_full['Length'].corr(df_full['Stroke_Volume_Ventricle_nl']))# calculate correlation
print("Length vs Cardiac_Output_nl_per_min:", df_full['Length'].corr(df_full['Cardiac_Output_nl_per_min'])) # calculate correlation

In [ ]:
corr = df_full.corr(method ='pearson') # calculate pearson correlation
corr 

In [ ]:
corr_volume = corr.drop(columns=corr.columns[4:12]) # drop columns and only keep volume
corr_volume = corr_volume[0:4] # keep only first 4 rows
corr_volume # show dataframe

In [ ]:
corr_function = corr.drop(columns=corr.columns[0:4]) # drop columns and only keep function
corr_function = corr_function[5:10] # keep only rows 5 to 10
corr_function # show dataframe

In [ ]:
corr_significant = corr[(corr>0.7) | (corr< -0.7)] # keep only significant correlations
corr_significant # show dataframe
corr_table=corr_significant # create new dataframe
corr_table.to_csv("correlation-table.csv") # save dataframe as csv file